In [ ]:
# Dependencies.
import parameterized_transforms.core as ptc
import parameterized_transforms.transforms as ptx
import parameterized_transforms.utils as ptu

import torchvision.transforms.functional as tv_fn

import numpy as np

import PIL.Image as Image
import PIL.ImageChops as ImageChops

import typing as t

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class RandomColorErasing(ptc.AtomicTransform):

    def __init__(
        self,
        tx_mode: ptc.TransformMode = ptc.TransformMode.CASCADE,
        default_params_mode: ptc.DefaultParamsMode = ptc.DefaultParamsMode.RANDOMIZED,
    ) -> None:
        """The initializer.

        :param tx_mode: The mode of the transform.
            DEFAULT: `ptc.TransformMode.CASCADE`.
            SUPPORTS: `ptc.TransformMode` enum type.
        :param default_params_mode: The mode of the default (identity) params.
            DEFAULT: `ptc.DefaultParamsMode.RANDOMIZED`.
            SUPPORTS: `ptc.DefaultParamsMode` enum type.
        """

        super(RandomColorErasing, self).__init__()

        self.tx_mode = tx_mode  # Alternatively, pass tx_mode=tx_mode to the above super call.
        self.default_params_mode = default_params_mode

        self.param_count = self.set_param_count()  # Recommended.

    def set_param_count(self) -> int:
        """Returns the number of parameters for the transform.

        :return: The number of parameters.

        We encode the rectangle to be erased using 4 integers `i, j, h, w`.
        The tuple `(i, j)` encodes the top-left point of the crop
            (along the height and the width respectively).
        The integers `h, w` are the crop height and crop width.
        We encode the fill-color in terms of 3 integers in range `[0, 255]`
            representing the `r, g, b` components.

        Thus, in total, there are 7 parameters.
        """
        return 7

    def get_raw_params(self, img: ptc.IMAGE_TYPE) -> ptc.PARAM_TYPE:
        """Get raw parameters to decide which rectangle to erase and what
        should be the fill value.

        :param img: The input image.

        :return: The parameters for the rectangle to be erased and the fill value.
        """

        # Get a randomly sampled fill color.
        fill_color = np.random.randint(
            low=0, high=256, size=[3, ]
        ).astype(np.uint8)

        # Get the image dimensions.
        W, H = tv_fn.get_image_size(img=img)
        # Select two heights for the crop.
        i1, i2 = np.random.randint(low=0, high=H, size=[2, ]).tolist()
        j1, j2 = np.random.randint(low=0, high=W, size=[2, ]).tolist()
        # Get the location and size of the rectangle to be erased.
        i, h = min(i1, i2), max(abs(i2 - i1), 1)
        j, w = min(j1, j2), max(abs(j2 - j1), 1)

        # Return the tuple of raw parameters.
        return (i, j, h, w, fill_color)

    def apply_transform(
            self, img: ptc.IMAGE_TYPE, params: ptc.PARAM_TYPE, **kwargs
    ) -> ptc.IMAGE_TYPE:
        """Augments given data point using given parameters.

        :param img: The data point to be augmented.
        :param params: The raw parameters to be used for augmentation.

        :return: The augmented image.
        """

        # Unpack the raw parameters.
        i, j, h, w, fill_color = params

        # Conver the `PIL` image to `numpy`.
        img_np = np.asarray(img).astype(np.uint8)
        # Replace the rectangular region with the sampled color.
        img_np[i: (i + h), j: (j + w), :] = fill_color

        # Convert this processed image back to a `PIL.image`
        aug = Image.fromarray(img_np)

        return aug

    def post_process_params(
        self, img: ptc.IMAGE_TYPE, params: ptc.PARAM_TYPE
    ) -> ptc.PARAM_TYPE:
        """Post-processes the parameters of augmentations before outputting.

        :param img: The data point to be augmented.
        :param params: The raw local parameters to be post-processed.

        :return: The post-processed parameters.
        """

        # Unpack all parameters into a tuple of scalars.
        i, j, h, w, fill_color = params
        r, g, b = fill_color.tolist()

        return (i, j, h, w, r, g, b)

    def extract_params(
        self, params: ptc.PARAM_TYPE
    ) -> t.Tuple[ptc.PARAM_TYPE, ptc.PARAM_TYPE]:
        """Chunks the input parameters into two sets; the first required for
        the augmentation of the current data and the second to pass on to the
        next augmentations.

        :param params: The parameters remaining from the augmentations so far.

        :return: The tuple of the local and subsequent parameters.
        """

        return params[: self.param_count], params[self.param_count :]

    def pre_process_params(self, img: ptc.IMAGE_TYPE, params: ptc.PARAM_TYPE) -> ptc.PARAM_TYPE:
        """Pre-processes the parameters of augmentations after inputting.

        :param img: The data point to be augmented.
        :param params: The parameters from which to extract local parameters.

        :return: The pre-processed parameters ready for their usage.
        """

        # Unpack all parameters.
        i, j, h, w, r, g, b = params
        # Return the parameters as required for applying the transform.
        fill_color = np.array([r, g, b]).astype(np.uint8)

        # Return the raw parameters.
        return (i, j, h, w, fill_color)

    def get_default_params(self, img: ptc.IMAGE_TYPE, processed: bool = True) -> ptc.PARAM_TYPE:
        """Returns the parameters for preserving the input data information.

        :param img: The data point to be augmented.
        :param processed: Whether we want the processed default parameters.

        :return: The no-augmentation params for the class.
        """

        # If the default parameters are required to be unique, ...
        if self.default_params_mode == ptc.DefaultParamsMode.UNIQUE:

            # Define the rectangle to erase at the origin, with size 0.
            i, j, h, w = 0, 0, 0, 0
            # Define the fill color to be `(r, g, b) = (0, 0, 0)`.
            fill_color = np.zeros([3, ]).astype(np.uint8)

            raw_params = (i, j, h, w, fill_color)

        # Else, ...
        else:  # `self.default_params_mode == ptc.DefaultParamsMode.RANDOMIZED`

            # Select any point of the image as the rectangle location.
            W, H = tv_fn.get_image_size(img=img)
            i = int(np.random.randint(low=0, high=H))
            j = int(np.random.randint(low=0, high=W))
            # Set the size of the rectangle to 0.
            h, w = 0, 0

            # Select any fill color.
            fill_color = np.random.randint(
                low=0, high=256, size=[3, ]
            ).astype(np.uint8)

            raw_params = (i, j, h, w, fill_color)

        return (
            self.post_process_params(img=img, params=raw_params)
            if processed
            else raw_params
        )

    def __str__(self) -> str:
        """Defines the string representation of the transform.

        :return: The string representation of the transform.
        """
        return (
            f"RandomColorErasing("
            f"param_count={self.param_count}, "
            f"tx_mode={self.tx_mode}, "
            f"default_params_mode={self.default_params_mode}"
            f")"
        )

In [ ]:
class RandomSubsetApply(ptc.ComposingTransform):

    def __init__(
        self,
        transforms: t.List[t.Callable],
        tx_mode: ptc.TransformMode,
    ) -> None:
        """The initializer.

        :param transforms: The list of transforms.
        :param tx_mode: The transform mode.
        """

        super(RandomSubsetApply, self).__init__()

        self.tx_mode = tx_mode
        self.transforms = transforms  # All composing transforms will have a core transform set/list/tuple.

        self.param_count = self.set_param_count()

        # Set the start and end indices of core transform parameters.
        self.idx_limits = [0]
        for tx in self.transforms:
            self.idx_limits.append(self.idx_limits[-1] + tx.param_count)
        # Set the number of core transforms.
        self.num_core_transforms = len(self.transforms)

    def set_param_count(self) -> int:
        """Returns the total number of processed parameters generated by the
        transform under consideration.

        :return: The number of parameters for this transform.
        """

        # The total number of parameters is just addition of those of the components.
        return ptu.get_total_params_count(transforms=self.transforms)


    def get_raw_params(self, img: ptc.IMAGE_TYPE) -> ptc.PARAM_TYPE:
        """Generates the raw parameters used to augment current data point.

        :param img: The data point to be augmented.

        :return: Current raw parameters to augment the data point.
        """

        # Decide which transforms to apply and which to skip as 1's and 0's.
        return tuple(
            np.random.randint(low=0, high=2, size=[self.num_core_transforms])
        )

    def apply_cascade_transform(
        self, img: ptc.IMAGE_TYPE, params: ptc.PARAM_TYPE, **kwargs
    ) -> ptc.TRANSFORM_RETURN_TYPE:
        """Augments given data point using given parameters.

        :param img: The data point to be augmented.
        :param params: The parameters to be used for augmentation.

        :return: The augmented image and corresponding augmentation parameters.
        """

        # Collect the augmentation params from the core transforms.
        aug_params = ()

        for apply, tx in zip(params, self.transforms):

            # If we have to apply the transform, apply and collect parameters.
            if apply:
                img, tx_params = tx(img=img, params=())
            # Else, get default parameters and apply them on the image.
            else:
                tx_params = tx.get_default_params(img=img, processed=True)
                img, rem_tx_params = tx.consume_transform(img=img, params=tx_params)
                assert rem_tx_params == ()  # Safety check!

            # Either way, collect the parameters.
            aug_params += tx_params

        # Return the final augmented image and the corresponding parameters.
        return img, aug_params

    def post_process_params(
        self,
        img: ptc.IMAGE_TYPE,
        params: ptc.PARAM_TYPE,
        aug_params: ptc.PARAM_TYPE
    ) -> t.Tuple[ptc.PARAM_TYPE, ptc.PARAM_TYPE]:
        """Post-processes the parameters of augmentations before outputting.

        :param img: The data point to be augmented.
        :param params: The raw local parameters to be post-processed.
        :param aug_params: The processed params from the composition of transforms.

        :return: The post-processed parameters.
        """

        # We ignore the local parameters describing which transforms to apply.
        return (), aug_params

    def extract_params(
        self, params: ptc.PARAM_TYPE
    ) -> t.Tuple[ptc.PARAM_TYPE, ptc.PARAM_TYPE]:
        """Chunks the input parameters into two sets; the first required for
        the augmentation of the current data and the second to pass on to the
        next augmentations.

        :param params: The parameters remaining from the augmentations so far.

        :return: The tuple of the local and subsequent parameters.
        """

        return params[: self.param_count], params[self.param_count :]

    def pre_process_params(
        self, img: ptc.IMAGE_TYPE, params: ptc.PARAM_TYPE
    ) -> t.Tuple[ptc.PARAM_TYPE, ptc.PARAM_TYPE]:
        """Pre-processes the parameters of augmentations after inputting.

        :param img: The data point to be augmented.
        :param params: The parameters from which to extract local parameters.

        :return: The pre-processed parameters ready for their usage.
        """

        # We do NOT need local params, the aug params contain all information.
        return (), params

    def apply_consume_transform(
        self,
        img: ptc.IMAGE_TYPE,
        params: ptc.PARAM_TYPE,
        aug_params: ptc.PARAM_TYPE
    ) -> ptc.TRANSFORM_RETURN_TYPE:
        """Applies the transform with given parameters by consuming them.

        :param img: The data point to be augmented.
        :param params: The parameters remaining from the augmentations so far.
        :param aug_params: The parameters from composition of transforms.

        :return: The augmented image with the remaining parameters.
        """

        # We ignore the local params and only chunk and apply aug_params.
        for idx, tx in enumerate(self.transforms):

            # Extract the parameters for the current transform.
            tx_params = aug_params[
                self.idx_limits[idx]:self.idx_limits[idx + 1]
            ]
            # Apply these params to the image.
            img, rem_tx_params = tx.consume_transform(img=img, params=tx_params)
            assert rem_tx_params == ()  # Safety check!

        # Return the transformed image and the remaining parameters.
        return img, ()

    def get_default_params(
        self, img: ptc.IMAGE_TYPE, processed: bool = True
    ) -> ptc.PARAM_TYPE:
        """Returns the parameters for preserving the input data information.

        :param img: The data point to be augmented.
        :param processed: Whether we want the processed default parameters.

        :return: The no-augmentation params for the class.
        """

        # Get local params.
        local_params = (0 for _ in range(self.num_core_transforms))  # No transform to apply.

        # Collect the core default parameters.
        core_default_params = ()

        # Get core default parameters and keep on applying.
        for tx in self.transforms:

            # Get the default parameters of the core transform.
            default_tx_params = tx.get_default_params(img=img, processed=processed)
            # Append the parameters to core default parameters.
            core_default_params += default_tx_params

            # To make default parameters from the next transforms consistent, apply these default parameters too.
            img, rem_tx_params = tx.consume_transform(img=img, params=default_tx_params)
            assert rem_tx_params == ()  # Safety check!

        return (
            self.concat_params(
                *(self.post_process_params(img=img, params=local_params, aug_params=core_default_params))
            )
            if processed
            else local_params + core_default_params
        )

    def __str__(self) -> str:
        """Defines the string representation of the transform.

        :return: The string representation of the transform.
        """

        # Set the start and end strings.
        start_string, end_string = "RandomSubsetApply(", ")"

        # Set the local details of the `RandomSubsetApply` transform.
        details_string = ptu.indent(
            data = (
                f"param_count={self.param_count},\n"
                f"tx_mode={self.tx_mode}, "
            ), indentor="  ", connector="\n"
        )

        # Get the cleaned string representation of the core transforms.
        core_start_string, core_end_string = "transforms=(", ")"

        core_tx_strings = []
        for tx in self.transforms:
            core_tx_strings.append(
                ptu.indent(data=str(tx), indentor="  ", connector="\n")
            )
        core_tx_string = ptu.indent(data=",\n".join(core_tx_strings), indentor="  ", connector="\n")
        core_combined_string = ptu.indent(
            data="\n".join([core_start_string, core_tx_string, core_end_string]),
            indentor="  ",
            connector="\n"
        )

        return "\n".join([start_string, details_string, core_combined_string, end_string])

In [ ]:
def show_image(
    data: Image.Image, title: str, params: t.Any = (), save: bool = False,
) -> None:
    """Helper function to visualize the images.

    :param data: The image data of type `PIL.Image.Image`.
    :param title: The title of the image.
    :param params: The parameters for the image.
        DEFAULT: `()`.
    :param save: Whether to save the image.
        DEFAULT: `True`.
    """

    # Set figure style.
    plt.style.use("dark_background")
    fig = plt.figure(figsize=(10, 10))

    # Process the data.
    data = np.asarray(data).astype(np.uint8)
    assert 0.0 <= np.min(data) <= np.max(data) <= 255.0

    # Clean the parameters.
    params = tuple(
        (int(elt) if elt == np.floor(elt) else round(elt, 2)) for elt in params
    )

    # Plot the image.
    plt.imshow(data)
    plt.title(f"{title}\nparams = {params}", fontdict={"size": 15})
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.savefig(f"{title}.png")
    plt.show(block=False)

In [ ]:
# Load the cat and the dog images.
cat_img = Image.open("../assets/cat.jpg")
width, height = cat_img.size
print(f"INFO | cat image dimensions: (width={width}, height={height})")
dog_img = Image.open("../assets/dog.jpg")
width, height = dog_img.size
print(f"INFO | dog image dimensions: (width={width}, height={height})")
# Show the images.
show_image(data=cat_img, title="Original Cat Image", params=())
show_image(data=dog_img, title="Original Dog Image", params=())
plt.close("all")

In [ ]:
#--------------------------------------------------------------------------------

# Create and display the transform.
tx_RCE = RandomColorErasing(
    tx_mode=ptc.TransformMode.CASCADE,
    default_params_mode=ptc.DefaultParamsMode.RANDOMIZED
)
print("-"*79)
print(tx_RCE)

#--------------------------------------------------------------------------------

# Sample augmentations.
for idx in range(5):
    aug, params = tx_RCE(img=cat_img, params=())  # Same as `tx_RCE.cascade_transform` because `tx_mode=ptc.TransformMode.CASCADE` is set.
    show_image(data=aug, title=f"RandomColorErasing Augmented Image", params=params)
plt.close("all")

#--------------------------------------------------------------------------------

# Check consistency! Get an augmented image with its parameters and push the parameters into the transform in CONSUME mode!
aug_cascade, params_cascade = tx_RCE(img=cat_img, params=())  # Same as `tx_RCE.cascade_transform(img=cat_img, params=())`
aug_consume, params_remaining = tx_RCE.consume_transform(img=cat_img, params=params_cascade)
assert params_remaining == ()
show_image(data=aug_cascade, title="RandomColorErasing Augmented Image", params=params_cascade)
plt.close("all")
# The augmentation generated with given parameters matches the original augmentation, as it should!
print("-"*79)
print(f"INFO | augmented images match: {ImageChops.difference(aug_cascade, aug_consume).getbbox() is None}")

#--------------------------------------------------------------------------------

# Applying default parameters.
default_params = tx_RCE.get_default_params(img=cat_img, processed=True)
# Apply the default parameters to the image.
default_aug_image, rem_params = tx_RCE.consume_transform(img=cat_img, params=default_params)
assert rem_params == ()
show_image(data=default_aug_image, title="RandomColorErasing Default Augmented Image", params=default_params)
plt.close("all")
# Check if in this case, the original image matches the one obtained by applying default parameters.
print("-"*79)
print(f"INFO | default augmented image matches the original image: {ImageChops.difference(default_aug_image, cat_img).getbbox() is None}")

In [ ]:
#--------------------------------------------------------------------------------

# Create and display the transform.
tx_RSA = RandomSubsetApply(
    transforms=[
        RandomColorErasing(
            tx_mode=ptc.TransformMode.CASCADE,
            default_params_mode=ptc.DefaultParamsMode.RANDOMIZED
        ),
        ptx.RandomRotation(degrees=30)  # Rotation by an angle `A` degrees such that `A` ~ [-30, 30].
    ],
    tx_mode=ptc.TransformMode.CASCADE
)

print("-"*79)
print(tx_RSA)

#--------------------------------------------------------------------------------

# Sample augmentations.
for idx in range(5):
    aug, params = tx_RSA(img=dog_img, params=())  # Same as `tx_RSA.cascade_transform` because `tx_mode=ptc.TransformMode.CASCADE` is set.
    show_image(data=aug, title=f"RandomSubsetApply Augmented Image", params=params)
plt.close("all")

#--------------------------------------------------------------------------------

# Check consistency! Get an augmented image with its parameters and push the parameters into the transform in CONSUME mode!
aug_cascade, params_cascade = tx_RSA(img=dog_img, params=())  # Same as `tx_RSA.cascade_transform(img=cat_img, params=())`
aug_consume, params_remaining = tx_RSA.consume_transform(img=dog_img, params=params_cascade)
assert params_remaining == ()
show_image(data=aug_cascade, title="RandomSubsetApply Augmented Image", params=params_cascade)
plt.close("all")
# The augmentation generated with given parameters matches the original augmentation, as it should!
print("-"*79)
print(f"INFO | augmented images match: {ImageChops.difference(aug_cascade, aug_consume).getbbox() is None}")

#--------------------------------------------------------------------------------

# Applying default parameters.
default_params = tx_RSA.get_default_params(img=dog_img, processed=True)
# Apply the default parameters to the image.
default_aug_image, rem_params = tx_RSA.consume_transform(img=dog_img, params=default_params)
assert rem_params == ()
show_image(data=default_aug_image, title="RandomSubsetApply Default Augmented Image", params=default_params)
plt.close("all")
# Check if in this case, the original image matches the one obtained by applying default parameters.
print("-"*79)
print(f"INFO | default augmented image matches the original image: {ImageChops.difference(default_aug_image, dog_img).getbbox() is None}")